In [1]:
import pandas as pd
import numpy as np

import math
import scipy.optimize as opt


In [2]:
# Data: Close
db = pd.read_excel('Data.xlsx', index_col = "Date")

# MSD

In [3]:
db.index = pd.to_datetime(db.index)
MSD_df = db['2006':'2013']
MSD_df = (MSD_df-MSD_df.mean())/MSD_df.std() #change rolling
MSD_df = MSD_df.dropna(axis = 1) #drop nan

In [4]:
target = {}
for i in range(0,len(MSD_df.columns)-1):
    for j in range(i+1,len(MSD_df.columns)):
        tmp = ((MSD_df.iloc[:,i] - MSD_df.iloc[:,j])**2).sum()
        target[str(MSD_df.columns[i])+','+str(MSD_df.columns[j])] = tmp

In [5]:
target_df = pd.DataFrame.from_dict(target, orient='index')
target_df = target_df.sort_values(by = 0)[0:10]
target_df

,0
"DIS,TRV",106.019533
"IBM,MCD",174.553811
"AAPL,IBM",183.653680
"CVX,KO",205.031258
"IBM,KO",225.295065
"DIS,HD",264.348677
"HD,TRV",282.786437
"PEP,PG",298.596370
"DIS,MMM",300.548727
"GOOGL,TRV",304.718140


In [6]:
pair1_df_train = db.loc["2006":"2013",target_df.index[0].split(",")].pct_change(1).dropna()
pair1_df_test = db.loc["2014":"2014",target_df.index[0].split(",")].pct_change(1).dropna()
pair1_df_train

,DIS,TRV
Date,,
2006-01-04,-0.016803,0.011089
2006-01-05,0.017507,0.009677
2006-01-06,0.013519,0.005538
2006-01-09,0.010509,0.000424
2006-01-10,0.012800,-0.007199
...,...,...
2013-12-24,0.007778,0.003701
2013-12-26,0.010427,0.005811
2013-12-27,-0.003618,-0.001778


In [7]:
# ytlist = pair1_df_train.iloc[:,0] - pair1_df_train.iloc[:,1]

In [8]:
from scipy.stats import t

# ytlist = [] #用來收集過去的yt們，算z_td、s_z時會用到

# #Table1 參數
# fi_1_0 = 0.1046
# fi_1_1 = 0.3886
# fi_2_0 = 0.0961
# fi_2_1 = -0.2403
# alpha_1_0 = 0.1639
# alpha_1_1 = 0.2132
# beta_1_1 = 0.6631
# alpha_2_0 = -0.1035
# alpha_2_1 = -0.1063
# beta_2_1 = -0.1751
# v = 7
# gamma = 5.5563
# c_1 = -0.3569
# c_2 = 0.3136
# d = 1
# z_td = yt[len(yt)-d:-1] #t-d
# s_z = std(z_td)

# Fdis = (1+np.exp(-gamma*(z_td - c_1)(z_td - c_2))/s_z)**(-1)
# h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
# h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
# miu_1 = fi_1_0 + fi_1_1*y_t1
# miu_2 = fi_2_0 + fi_2_1*y_t1
# h_t = h_1_t + Fdis*h_2_t
# epsilon = t.rvs(v, size=1) #t-distribution
# a_t = h_t**0.5*epsilon #error term
# y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH


In [9]:
z_td = pair1_df_train.iloc[0:30,0]-pair1_df_train.iloc[0:30,1]
z_td

Date
2006-01-04   -0.027893
2006-01-05    0.007830
2006-01-06    0.007981
2006-01-09    0.010086
2006-01-10    0.019999
2006-01-11   -0.006730
2006-01-12    0.009006
2006-01-13    0.004873
2006-01-17    0.004903
2006-01-18   -0.002139
2006-01-19    0.046898
2006-01-20    0.000427
2006-01-23   -0.019551
2006-01-24    0.023248
2006-01-25   -0.007702
2006-01-26   -0.047477
2006-01-27   -0.007359
2006-01-30    0.021383
2006-01-31    0.012919
2006-02-01   -0.006649
2006-02-02    0.000069
2006-02-03    0.007673
2006-02-06    0.012736
2006-02-07    0.077869
2006-02-08    0.009547
2006-02-09   -0.001655
2006-02-10   -0.005409
2006-02-13   -0.002098
2006-02-14   -0.011618
2006-02-15    0.014558
dtype: float64

In [10]:
fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, v, gamma, c_1, c_2 = np.array([0.1046, 0.3886, 0.0961, -0.2403, 0.1639, 0.2132, 0.6631, -0.1035, -0.1063, -0.1751, 7.1122, 5.5563, -0.3569, 0.3136])
tmp = 30
gamma = 5.5563
c_1 = -0.3569
c_2 = 0.3136
z_td = pair1_df_train.iloc[0:tmp,0] - pair1_df_train.iloc[0:tmp,1]
s_z = z_td.std()   
Fdis = (1+np.exp(-gamma*(z_td - c_1)*(z_td - c_2)/s_z))**(-1)
Fdis

Date
2006-01-04    3.701233e-13
2006-01-05    4.572923e-13
2006-01-06    4.583363e-13
2006-01-09    4.736674e-13
2006-01-10    5.701199e-13
2006-01-11    3.878527e-13
2006-01-12    4.656085e-13
2006-01-13    4.384032e-13
2006-01-17    4.385799e-13
2006-01-18    4.037920e-13
2006-01-19    1.213226e-12
2006-01-20    4.149179e-13
2006-01-23    3.668781e-13
2006-01-24    6.124634e-13
2006-01-25    3.850881e-13
2006-01-26    4.343492e-13
2006-01-27    3.860391e-13
2006-01-30    5.873985e-13
2006-01-31    4.968918e-13
2006-02-01    3.880919e-13
2006-02-02    4.132669e-13
2006-02-03    4.562162e-13
2006-02-06    4.952965e-13
2006-02-07    4.569811e-12
2006-02-08    4.695928e-13
2006-02-09    4.057628e-13
2006-02-10    3.919414e-13
2006-02-13    4.039553e-13
2006-02-14    3.759850e-13
2006-02-15    5.117912e-13
dtype: float64

In [11]:
def fun_MLE(p):
    fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, v, gamma, c_1, c_2= p
    # Define h_t1
    tmp = 30
    target = 1
    z_td = pair1_df_train.iloc[0:tmp,0]-pair1_df_train.iloc[0:tmp,1]
    z_td = z_td.to_numpy()
    s_z = z_td.std()
    Fdis = (1+np.exp(-gamma*(z_td - c_1)*(z_td - c_2))/s_z)**(-1)
    h_1_t = pair1_df_train.iloc[0:tmp,0].std()
    h_2_t = pair1_df_train.iloc[0:tmp,1].std()
    h_t1 = h_1_t + Fdis*h_2_t
    epsilon = t.rvs(v, size=1) #t-distribution
    a_t1 = h_t1**0.5*epsilon #error term


    y_t1 = pair1_df_train.iloc[tmp-1,0] - pair1_df_train.iloc[tmp-1,1]
    # Define sigma, real_y_t, y_t
    for i in range(tmp,len(pair1_df_train)):
        z_td = pair1_df_train.iloc[0:i,0]-pair1_df_train.iloc[0:i,1]
        z_td = z_td.to_numpy()
        s_z = z_td.std()
        real_y_t = pair1_df_train.iloc[i,0] - pair1_df_train.iloc[i,1]
        # real_y_t = real_y_t.to_numpy()
        h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
        h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
        miu_1 = fi_1_0 + fi_1_1*y_t1
        miu_2 = fi_2_0 + fi_2_1*y_t1
        h_t = h_1_t + Fdis*h_2_t
        epsilon = t.rvs(v, size=1) #t-distribution
        a_t = h_t**0.5*epsilon #error term
        y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH

        h_t1 = h_t
        a_t1 = a_t
        y_t1 = real_y_t

        sigma = h_t
        target *= (2*np.pi*sigma**2)**(-0.5)*np.exp(-(real_y_t-y_t)**2/2*sigma**2) #MLE
    target = -target
    return target

In [13]:
#MLE

initial = np.array([0.10, 0.4, 0.10, -0.25, 0.15, 0.2, 0.7, -0.1, -0.1, -0.2, 7, 5, -0.35, 0.3])  # 後面兩項問號
minimum = opt.fmin(func = fun_MLE, x0 = initial)
minimum

ValueError: setting an array element with a sequence.

# Trade

In [ ]:
df_return = db.pct_change()

In [ ]:
global results

In [ ]:
results = {'signals':[],
           'returns':[],
           'A_SoldPrices':[],
           'A_BoughtPrices':[],
           'B_SoldPrices':[],
           'B_BoughtPrices':[]
           }

In [ ]:
# Input1 : Real return spread
# Input2 : Estimated return spread
# Output : Gain, Trading Signal

y1 = df_return.iloc[-1,0] 
y2 = df_return.iloc[-1,1]
p1 = db.iloc[-1:,0] #A Price
p2 = db.iloc[-1,1] #B Price

#First trade
i = 0

#signal
tmp0 = 0
tmp1 = 1
tmp2 = -1

#First trade
if i == 0:
    if (y1 > y2):
        signal = tmp1
        results['A_SoldPrices'].append(p1)
        results['B_BoughtPrices'].append(p2)
        i = 1
        tmp = 0
    
    elif (y1 < y2):  
        signal = tmp2
        results['A_BoughtPrices'].append(p1)
        results['B_SoldPrices'].append(p2) 
        i = 1
        tmp = 0
    else:
        signal = tmp0
        i = 0
        tmp = 1

elif i == 1:
    # After first trade
    if (y1 > y2) & (results['signals'][-1] == tmp1) :
        signal = tmp1
        results['A_SoldPrices'].append(y1)
        results['B_BoughtPrices'].append(y2)
        tmp = -np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])+np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1])
  
    elif (y1 <y2) & (results['signals'][-1] == tmp2):  
        signal = tmp2
        results['A_BoughtPrices'].append(y1)
        results['B_SoldPrices'].append(y2)
        tmp = np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])-np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1]) 

    else:
        signal = tmp0
        tmp = 0


results['signals'].append(signal)
results['returns'].append(tmp)